In [1]:
import numpy as np
import pandas as pd
import copy
import datetime as dt
import argparse
import netCDF4 as nc
from scipy import stats
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import matplotlib.colors as mcolors
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import xarray 
import matplotlib as mpl
import esmpy as ESMF
import xesmf as xe

In [2]:
def outline_nep_domain(ax,clon,clat):
    #NEP DOMAIN OUTLINE
    ax.plot(clon[0,:],clat[0,:],linewidth=1,color='k',transform=ccrs.PlateCarree(),zorder=42)
    ax.plot(clon[:,0],clat[:,0],linewidth=1,color='k',transform=ccrs.PlateCarree(),zorder=42)
    ax.plot(clon[-1,:],clat[-1,:],linewidth=1,color='k',transform=ccrs.PlateCarree(),zorder=42)
    ax.plot(clon[:,-1],clat[:,-1],linewidth=1,color='k',transform=ccrs.PlateCarree(),zorder=42)

In [3]:
def nep_mask(clon,clat):
    sourcegrid = ESMF.Grid(np.array(nep_lon.shape), staggerloc = ESMF.StaggerLoc.CORNER,coord_sys = ESMF.CoordSys.SPH_DEG)
    sourcegrid.add_item(ESMF.GridItem.MASK,[ESMF.StaggerLoc.CENTER])
    grid_mask = sourcegrid.get_item(ESMF.GridItem.MASK)
    grid_mask[...] = lsm.astype(np.int32)

    source_lon = sourcegrid.get_coords(0, staggerloc=ESMF.StaggerLoc.CORNER)
    source_lat = sourcegrid.get_coords(1, staggerloc=ESMF.StaggerLoc.CORNER)

    source_lon[...] = nep_clon
    source_lat[...] = nep_clat

    sourcefield = ESMF.Field(sourcegrid, name = 'lsm')
    srcfracfield = ESMF.Field(sourcegrid, 'srcfracfield')

    Xn, Yn = np.meshgrid(clon,clat)
    destgrid = ESMF.Grid(np.array(Xn[1:,1:].shape), staggerloc = ESMF.StaggerLoc.CORNER, coord_sys = ESMF.CoordSys.SPH_DEG)

    dest_lon = destgrid.get_coords(0,staggerloc=ESMF.StaggerLoc.CORNER)
    dest_lat = destgrid.get_coords(1,staggerloc=ESMF.StaggerLoc.CORNER)

    dest_lon[...] = Xn
    dest_lat[...] = Yn

    destfield = ESMF.Field(destgrid, name = 'comp_grid')

    # DEFINE INTERPOLATION FUNCTION
    regrid = ESMF.Regrid(sourcefield, destfield,regrid_method = ESMF.RegridMethod.CONSERVE,
                     src_mask_values=np.array([0], dtype=np.int32),src_frac_field=srcfracfield,
                     norm_type=ESMF.NormType.FRACAREA,unmapped_action = ESMF.UnmappedAction.IGNORE)
    const_offset = np.nanmax(lsm) + 10
    sourcefield.data[...] = lsm + const_offset

    destfield = regrid(sourcefield, destfield)

    lsm_out = copy.deepcopy(destfield.data) - const_offset
    lsm_out[lsm_out>.5]=1
    lsm_out[lsm_out<.5]=0 
    
    return lsm_out

In [4]:
def center_to_outer(center, left=None, right=None):
    """
    Given an array of center coordinates, find the edge coordinates,
    including extrapolation for far left and right edge.
    """
    edges = 0.5 * (center.values[0:-1] + center.values[1:])
    if left is None:
        left = edges[0] - (edges[1] - edges[0])
    if right is None:
        right = edges[-1] + (edges[-1] - edges[-2])
    outer = np.hstack([left, edges, right])
    return outer

In [5]:
def corners(lon, lat):
    """
    Given 1D lon and lat, return 1D lon and lat corners 
    for use in pcolormesh or xesmf conservative
    """
    lonc = center_to_outer(lon)
    latc = center_to_outer(lat)
    assert len(lonc) == len(lon) + 1
    assert len(latc) == len(lat) + 1
    return lonc, latc

In [6]:
def calc_stats():
    cop_vals = copepod_season
    
    log_cop_vals = np.log10(cop_vals)
    log_nep_vals = np.log10(nep_vals)
    
    nas = np.logical_or(np.isnan(log_cop_vals), np.isnan(log_nep_vals))
    
    mean_bias = np.nansum(((log_nep_vals[~nas]*area[~nas])-(log_cop_vals[~nas]*area[~nas]))/np.nansum(area[~nas]))
    print('AREA-WEIGHTED MEAN BIAS:',mean_bias)

    medae = np.nanmedian(np.abs(log_nep_vals[~nas]-log_cop_vals[~nas]))
    
    rmse = np.sqrt(np.sum((((log_nep_vals[~nas]-log_cop_vals[~nas])**2)*area[~nas])/np.sum(area[~nas])))
    print('AREA-WEIGHTED RMSE:', rmse)
    
    corr = stats.pearsonr(log_cop_vals[~nas], log_nep_vals[~nas])
    print(corr)
    
    return mean_bias, rmse, medae, corr[0]

In [7]:
# NEP Grid
nep_grd_fil = '/work/role.medgrp/NEP/plotting/shared_files/NEP_ocean_static_nomask.nc'
grd_fid = nc.Dataset(nep_grd_fil)

# Extracting tracer lat/lon from the supergrid
nep_lat = grd_fid.variables['geolat'][:]
nep_lon = grd_fid.variables['geolon'][:]

# Extracting tracer corner lat/lon from the supergrid - needed for regridding and pcolor plots
nep_clat = grd_fid.variables['geolat_c'][:]
nep_clon = grd_fid.variables['geolon_c'][:]

depth = grd_fid.variables['deptho'][:]

lsm = grd_fid.variables['wet'][:]
lsm[lsm<1]=0 
lsm[lsm>1]=1 

nep_area = grd_fid.variables['areacello'][:].squeeze()
nep_area[lsm<.5]=0

In [8]:
#Copepod grid for NEP
cop_lon = np.arange(150.125,260,0.25)
cop_clon = np.arange(150,260.25,0.25)

cop_lat = np.arange(10.125,81.5,0.25) 
cop_clat = np.arange(10,81.6,0.25) 

cop_lons, cop_lats = np.meshgrid(cop_lon,cop_lat)
cop_dir = '/work/role.medgrp/NEP/external_data/COPEPOD/copepod-2012__biomass-fields/data/'

In [ ]:
PC = ccrs.PlateCarree(central_longitude=-100)

pp_root = '/work/role.medgrp/NEP/plotting/Figure_11/'
seasonal_obs = []
for i, s in enumerate(['DJF', 'MAM', 'JJA', 'SON']):
    season_data = pd.read_csv(f'/work/role.medgrp/NEP/plotting/Figure_11/copepod-2012__cmass-m{i+13}-qtr.csv', index_col=['Latitude', 'Longitude'])
    season_data = season_data['Total Carbon Mass (mg-C/m3)'].to_xarray()
    season_data['season'] = s
    seasonal_obs.append(season_data)
        
seasonal_obs = xarray.concat(seasonal_obs, dim='season')
seasonal_obs = seasonal_obs.rename({'Latitude': 'lat', 'Longitude': 'lon'})
seasonal_obs *= 2.0 # correction for underestimation
cope_lons,cope_lats = np.meshgrid(seasonal_obs.lon,seasonal_obs.lat)
chuk_mask = cope_lats>66

xcorner, ycorner = corners(seasonal_obs.lon, seasonal_obs.lat)
xcorner[xcorner<0] = xcorner[xcorner<0]+360
nep_mask_val = nep_mask(xcorner,ycorner)

mesozoo = xarray.open_dataset((pp_root + 'nep_mesozoo_200_1993-2019_ts.nc'))['mesozoo_200']
mesozoo_climo = mesozoo.groupby('time.season').mean('time').load() * (106 / 16) * 12 * 1000 # mg C
mesozoo_vol = mesozoo_climo / np.clip(depth, None, 200)

# Regridder details for statistics
mesozoo_ds = mesozoo_vol.to_dataset(name = 'mesozoo_vol')
mesozoo_ds = mesozoo_ds.assign_coords(lon = (('yh', 'xh'), nep_lon))
mesozoo_ds = mesozoo_ds.assign_coords(lat = (('yh', 'xh'), nep_lat))
mesozoo_ds = mesozoo_ds.assign_coords(lat_b = (('yq', 'xq'), nep_clat))
mesozoo_ds = mesozoo_ds.assign_coords(lon_b = (('yq', 'xq'), nep_clon))
mesozoo_ds['area'] = (('yh', 'xh'), nep_area)

copepod_ds = seasonal_obs.to_dataset()
copepod_ds = copepod_ds.assign_coords(lat_b = (('lat_b'), ycorner))
copepod_ds = copepod_ds.assign_coords(lon_b = (('lon_b'), xcorner))

regridder = xe.Regridder(mesozoo_ds, copepod_ds, "conservative")
dr_out = regridder(mesozoo_ds)
area = dr_out.area.values

# Figure details
cmap = mpl.cm.RdYlBu_r
norm = mcolors.LogNorm(vmin=1,vmax=200)
common = dict(cmap=cmap, norm=norm)
stats_box_props = dict(boxstyle="square", fc="w", ec="0.0", alpha=1)

heights = [3,3,3,1]
fig = plt.figure(figsize=(16,9),dpi=300)
spec = fig.add_gridspec(ncols=4, nrows=4,hspace=0.0,wspace=0.01,height_ratios=heights)#,width_ratios=widths)
lab_x = -0.25
lab_y = 0.5
for col, s in enumerate(['DJF', 'MAM', 'JJA', 'SON']):
    print(s)
    
    # mom6
    nep_zoop = copy.deepcopy(mesozoo_vol.sel(season=s).values)
    nep_zoop[nep_lat>66]=np.nan
    ax = fig.add_subplot(spec[0,col], projection=ccrs.PlateCarree(central_longitude=-100))
    p = ax.pcolormesh(nep_clon, nep_clat, nep_zoop, transform=ccrs.PlateCarree(), **common)
    ax.set_title(s,fontsize=20)
    if col == 0:
        ax.text(lab_x,lab_y,'NEP10k',ha="center", va="center",fontsize=18,transform=ax.transAxes)
        
    # copepod
    copepod_season = seasonal_obs.sel(season=s).values
    copepod_season[nep_mask_val==0]=np.nan
    copepod_season[chuk_mask]=np.nan
    ax = fig.add_subplot(spec[1,col], projection=ccrs.PlateCarree(central_longitude=-100))
    p = ax.pcolormesh(xcorner, ycorner, copepod_season, transform=ccrs.PlateCarree(), **common)
    if col == 0:
        ax.text(lab_x,lab_y,'COPEPOD',ha="center", va="center",fontsize=18,transform=ax.transAxes)
        
    # stats
    # NEP Remapped
    nep_vals = dr_out.mesozoo_vol.sel(season=s).values
    nep_vals[nep_vals<=0] = np.nan
    nep_vals[chuk_mask]=np.nan
    plot_nep_remap = copy.deepcopy(nep_vals)
    plot_nep_remap[np.isnan(copepod_season)]=np.nan
    ax = fig.add_subplot(spec[2,col], projection=ccrs.PlateCarree(central_longitude=-100))
    p = ax.pcolormesh(xcorner, ycorner, plot_nep_remap, transform=ccrs.PlateCarree(), **common)
    if col == 0:
        ax.text(lab_x,lab_y,'NEP10k\nRemapped',ha="center", va="center",fontsize=18,transform=ax.transAxes)
        
    bias,rmse,medae,corr = calc_stats()
    
    ax.text(.03, .05, ('Bias: ' + f'{bias:.2f}' + '\nRMSE: ' + f'{rmse:.2f}' + '\nMedAE: ' + f'{medae:.2f}' + '\nR: ' + f'{corr:.2f}'), 
            ha="left", va="bottom", size=13, 
            bbox=stats_box_props,transform=ax.transAxes)
    
n=0        
for ax in fig.axes:
    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.set_facecolor('#999999')
    ax.set_extent([np.min(nep_clon),np.max(nep_clon),
                   np.min(nep_clat),70], crs=ccrs.PlateCarree())
    
    land_50m = cfeature.NaturalEarthFeature('physical', 'land', '50m',facecolor='#999999')
    ax.add_feature(land_50m,zorder=40)
    ax.coastlines('50m',linewidth=0.1,zorder=40)
    outline_nep_domain(ax,nep_clon,nep_clat)
    
    
    # ADDING GRID LINES AND GRID LABELS
    gl = ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,y_inline=False)
    
    gl.xlocator = mticker.FixedLocator([180, -150, -120])
    gl.ylocator = mticker.FixedLocator([25, 55])
    
    gl.xlabel_style = {'size': 13}
    gl.ylabel_style = {'size': 13}

    gl.top_labels = False
    gl.bottom_labels = False
    gl.left_labels = False
        
    if n>8:
        # Latitude labels on right side need to be added manually
        for lat_lab in gl.ylocator.locs:
            ax.text(np.max(nep_clon)+2,lat_lab,(str(lat_lab)+ '$^{\circ}$N'),
                    transform=ccrs.PlateCarree(),fontsize=13,zorder=50,ha='left',va='center')
            
    if (n+1)%3 == 0:
        gl.bottom_labels = True
    
    if n == 2:
        pos = ax.get_position()
        cax = fig.add_axes([pos.x0, pos.y0-.4*pos.height, 4*pos.width, .1*pos.height])
        cbar = plt.colorbar(p,cax=cax,extend='max',orientation='horizontal')
        cbar.ax.set_xticks([1,2,5,10,20,50,100,200])
        cbar.ax.set_xticklabels([1,2,5,10,20,50,100,200])
        cbar.ax.tick_params(labelsize=13)    
        cbar.ax.set_title('mesozooplankton biomass (mg C m$^{-3}$)',y=-4,fontsize=18)
        
    plt.setp(ax.spines.values(), linewidth=1,zorder=100)
    n+=1

plt.savefig('Figure11_2xCopepod_1993-2019')

DJF
AREA-WEIGHTED MEAN BIAS: 0.1369759012565539
AREA-WEIGHTED RMSE: 0.39390147010600673
PearsonRResult(statistic=0.4448242990971937, pvalue=2.4826114876102683e-60)
MAM
AREA-WEIGHTED MEAN BIAS: -0.08199661040852273
AREA-WEIGHTED RMSE: 0.49728631326823447
PearsonRResult(statistic=0.35502658998964065, pvalue=8.267309499688314e-72)
JJA
AREA-WEIGHTED MEAN BIAS: 0.020354300038601288
AREA-WEIGHTED RMSE: 0.4858681664585275
PearsonRResult(statistic=0.4491095107409968, pvalue=2.5295672531208076e-170)
SON
AREA-WEIGHTED MEAN BIAS: 0.238469688340869
AREA-WEIGHTED RMSE: 0.5103174453247583
PearsonRResult(statistic=0.30022138715281926, pvalue=8.307544466368552e-35)
